Convert the text to vecor and built classification (to categorys) model

 Import librarys

In [4]:
import pandas as pd
import gensim
import numpy as np
import io
import requests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

 Import data

In [5]:
url = "https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

First look

In [6]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [7]:
print('Number of news items: {}'.format(len(df)))

Number of news items: 2225


Change data format (X)

In [11]:
def newFormat(df):
    newFormat = []
    for doc in df['text']:
        newFormat.append(gensim.utils.simple_preprocess(doc))
    return newFormat

In [12]:
documents = newFormat(df)

Initialize W2V model

In [13]:
model = gensim.models.Word2Vec(documents, size=100, window=10, min_count=2, workers=10)

Train model on CBOW fake task

In [14]:
model.train(sentences=documents, total_examples=len(documents), epochs=10)

(6435861, 8222420)

Get the vector- representation for one word

In [15]:
model.wv.get_vector("terror")

array([ 1.2694424 , -1.2860268 ,  0.9960562 ,  0.4304321 , -0.5830673 ,
        1.6549737 ,  1.4371467 ,  0.78013235, -0.30703086,  0.42572203,
       -0.4686204 ,  2.2567756 ,  1.2846935 , -0.85745126, -0.18723519,
       -1.8290185 , -0.69543964, -1.1084111 , -1.6052476 ,  1.7331901 ,
       -0.9258824 , -1.8807042 ,  0.5799423 ,  1.1048173 , -0.35119218,
        1.7125776 ,  0.3871888 ,  1.1183598 ,  0.9034464 ,  0.6501818 ,
       -0.896806  , -1.3414217 ,  0.6406782 , -2.346887  , -0.5761691 ,
        1.5720054 ,  1.5584743 , -0.29060698, -1.5814909 ,  1.0686527 ,
       -0.0812887 ,  0.25804007,  0.9246796 , -1.5086472 ,  0.7359569 ,
        2.832336  , -1.2045327 ,  0.20496014,  0.42071006,  0.23051572,
       -1.0013902 ,  0.5349644 ,  2.3389478 ,  0.37271836, -1.2090193 ,
       -1.5037785 , -0.20835379, -0.5712959 , -0.4233551 ,  0.4797269 ,
        0.8034948 ,  0.19000967,  0.5840332 , -0.63900435, -0.28269893,
       -0.640814  ,  1.8584152 ,  0.80373454,  2.1084657 ,  1.51

Get the 8 most similar words to a specific word 

In [16]:
model.wv.most_similar(positive="terror", topn=8)

[('suspects', 0.8642494082450867),
 ('terrorism', 0.8411095142364502),
 ('laws', 0.8354306221008301),
 ('detention', 0.8222179412841797),
 ('indefinitely', 0.8165493011474609),
 ('arrest', 0.7980394959449768),
 ('detained', 0.7937533855438232),
 ('detainees', 0.7868883609771729)]

Get the 6 most similar words to severl words

In [17]:
model.wv.most_similar(positive=["france", "germany"], topn=6)

[('australia', 0.7842113375663757),
 ('sweden', 0.7682937383651733),
 ('argentina', 0.7565438747406006),
 ('korea', 0.7378115653991699),
 ('brazil', 0.7374104261398315),
 ('republic', 0.7336969375610352)]

Odd one out

In [18]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["france","germany","sudan"])

C:\Users\user\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'sudan'

similarity = Distance -> Cosine similarity = Euclidean distance

In [19]:
model.wv.similarity(w1="dirty", w2="dirty")

1.0

Convert the documents with the W2V model

In [20]:
def document_to_vector(document, model):
    token_list = gensim.utils.simple_preprocess(document)
    vector_list = []
    for token in token_list:
        try:
            vector_list.append(model.wv[token])
        except KeyError:
            pass
    return np.average(vector_list, axis=0)

In [21]:
df['text'].iloc[0] #documents[0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [22]:
vec0 = document_to_vector(df['text'].iloc[0], model)
print(vec0.shape)

(100,)


In [23]:
print(vec0)

[ 0.15003632 -0.75395566  0.17867522 -0.64230347  0.51449496  0.22731787
 -0.00718398 -0.13092811  0.42770153 -0.3347693  -0.02076959  0.12693542
 -0.36243597  0.00699822 -0.4205801   0.09327846  0.14994851  0.16220164
 -0.19816859  0.01629554  0.1209539  -0.36463735 -0.01181827  0.50903076
 -0.34037295  0.42006797 -0.283955    0.22662675  0.05414711  0.3530428
 -0.14582583 -0.16319683 -0.6330359  -0.43890482  0.81061953 -0.31949076
 -0.08001008  0.09428914  0.68306     0.05342641  0.4777236   0.65019876
 -0.34958148  0.3567105   0.00893704 -0.52259237  0.564671   -0.37151492
  0.84422225 -0.15783632 -0.318606    0.04176624  0.13929068  0.4464985
  0.11834769  0.03823769  0.61221427 -0.04169255 -0.55333495  0.0108797
  0.20691508  0.03913882 -0.5352008  -0.08708108 -0.34518495  0.699256
  0.491659    0.6423476  -0.76250386  0.554575   -0.06557303  0.5419666
 -0.14993499 -0.27093503 -0.37840894 -0.46058667  0.35802484  0.11882226
 -0.29620522  0.51078784  0.46935576  0.03331386  0.24133

In [24]:
df['w2v'] = df['text'].apply(document_to_vector, model=model)

In [25]:
df.head()

,category,text,w2v
0,tech,tv future in the hands of viewers with home th...,"[0.15003632, -0.75395566, 0.17867522, -0.64230..."
1,business,worldcom boss left books alone former worldc...,"[0.31544575, 0.09478428, 0.03107781, -0.063333..."
2,sport,tigers wary of farrell gamble leicester say ...,"[0.07702949, -0.3854015, -0.18866463, -0.81674..."
3,sport,yeading face newcastle in fa cup premiership s...,"[0.14492513, -0.33911353, -0.03247725, -0.5256..."
4,entertainment,ocean s twelve raids box office ocean s twelve...,"[-0.17398588, 0.01132737, 0.05516599, 0.019544..."


Encode category (y)

In [26]:
lblenc = LabelEncoder()
df['lbl'] = lblenc.fit_transform(df['category'])

In [27]:
lblenc.classes_

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [28]:
df.head()

,category,text,w2v,lbl
0,tech,tv future in the hands of viewers with home th...,"[0.15003632, -0.75395566, 0.17867522, -0.64230...",4
1,business,worldcom boss left books alone former worldc...,"[0.31544575, 0.09478428, 0.03107781, -0.063333...",0
2,sport,tigers wary of farrell gamble leicester say ...,"[0.07702949, -0.3854015, -0.18866463, -0.81674...",3
3,sport,yeading face newcastle in fa cup premiership s...,"[0.14492513, -0.33911353, -0.03247725, -0.5256...",3
4,entertainment,ocean s twelve raids box office ocean s twelve...,"[-0.17398588, 0.01132737, 0.05516599, 0.019544...",1


Split data before model fitting

In [29]:
X = np.stack(df['w2v'].values)
y = df['lbl'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Train model

In [30]:
rndforest = RandomForestClassifier(n_estimators=50, max_depth=8)
rndforest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Test model

In [31]:
y_pred = rndforest.predict(X_test)
print(classification_report(y_test, y_pred, target_names=lblenc.classes_))

               precision    recall  f1-score   support

     business       0.95      0.92      0.94       181
entertainment       0.94      0.92      0.93       129
     politics       0.92      0.94      0.93       125
        sport       0.99      0.97      0.98       158
         tech       0.91      0.96      0.94       142

     accuracy                           0.94       735
    macro avg       0.94      0.94      0.94       735
 weighted avg       0.94      0.94      0.94       735

